03-filter_nodes_edges.ipynb
- Remove edges with nodes with no umls type or label
- Remove the following predicates: ['compared_with', 'higher_than', 'lower_than', 'different_from', 'different_than', 
'same_as','OCCURS_IN', 'PROCESS_OF', 'DIAGNOSES', 'METHOD_OF', 'USES','AUGMENTS', 'ADMINISTERED_TO', 'COMPLICATES']

In [3]:
import os
import pickle
%pylab
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
from collections import defaultdict
from tqdm import tqdm

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
edges = pd.read_csv('edges3.csv', index_col=0)
edges.head()

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID,NEG
379,C1412045,AFFECTS,C0020291,6298464,False
380,C1412045,AFFECTS,C0023946,20801151,False
381,C1412045,AFFECTS,C0028754,19789049,False
382,C1412045,AFFECTS,C0597304,1409557,False
383,C1412045,AFFECTS,C0599816,7617239,False


In [5]:
nodes = pd.read_csv("nodes_blm.csv", index_col=0)
nodes.head()

,ID,LABEL,umls_type,umls_type_label,blm_type
0,C0061133,gastrin releasing peptide (14-27),T116,"Amino Acid, Peptide, or Protein",protein
1,C1523610,"regulation of tube length, open tracheal system",T042,Organ or Tissue Function,biological_process_or_activity
2,C0312636,Antibody to hepatitis E virus,T116|T129,"Amino Acid, Peptide, or Protein|Immunologic Fa...",biological_entity
4,C0539817,cytochrome p30,T116|T126,"Amino Acid, Peptide, or Protein|Enzyme",protein
6,C0406240,Photosensitive atopic dermatitis,T047,Disease or Syndrome,disease_or_phenotypic_feature


In [6]:
print(len(nodes))
print(len(edges))

180491
22280924


In [7]:
# remove edges with no nodes
print(len(edges))
edges = edges[edges.SUBJECT_CUI.isin(nodes.ID) & edges.OBJECT_CUI.isin(nodes.ID)]
print(len(edges))

22280924
15378937


In [10]:
vc = edges.PREDICATE.value_counts()
vc

LOCATION_OF         1863675
INTERACTS_WITH      1850601
COEXISTS_WITH       1707696
AFFECTS             1443544
PART_OF             1231194
STIMULATES          1159243
INHIBITS             925338
ASSOCIATED_WITH      813132
CAUSES               716141
AUGMENTS             652698
TREATS               612205
PRODUCES             553511
DISRUPTS             471637
compared_with        351305
ISA                  247356
PREDISPOSES          230306
PREVENTS             117892
higher_than          107865
CONVERTS_TO           61644
PROCESS_OF            54726
MANIFESTATION_OF      36462
COMPLICATES           33992
USES                  25959
PRECEDES              24652
lower_than            21266
same_as               21083
OCCURS_IN             16317
ADMINISTERED_TO       13971
different_from         7959
DIAGNOSES              5378
METHOD_OF               189
Name: PREDICATE, dtype: int64

In [14]:
def drop_edges(df, drop_edges):
    idx = df.query('PREDICATE in @drop_edges').index
    df.drop(idx, inplace=True)

In [15]:
# remove edges we don't need
print(len(edges))
remove_edges = ['compared_with', 'higher_than', 'lower_than', 'different_from', 'different_than', 'same_as',
               'OCCURS_IN', 'PROCESS_OF', 'DIAGNOSES', 'METHOD_OF', 'USES',
               'AUGMENTS', 'ADMINISTERED_TO', 'COMPLICATES']
drop_edges(edges, remove_edges)
print(len(edges))

15378937
14066229


In [16]:
# remove nodes with no edges
print(len(nodes))
nodes = nodes[nodes.ID.isin(set(list(edges.SUBJECT_CUI) + list(edges.OBJECT_CUI)))]
print(len(nodes))

180491
165765


In [17]:
vc = edges.PREDICATE.value_counts()
vc

LOCATION_OF         1863675
INTERACTS_WITH      1850601
COEXISTS_WITH       1707696
AFFECTS             1443544
PART_OF             1231194
STIMULATES          1159243
INHIBITS             925338
ASSOCIATED_WITH      813132
CAUSES               716141
TREATS               612205
PRODUCES             553511
DISRUPTS             471637
ISA                  247356
PREDISPOSES          230306
PREVENTS             117892
CONVERTS_TO           61644
MANIFESTATION_OF      36462
PRECEDES              24652
Name: PREDICATE, dtype: int64

In [18]:
edges.to_csv('edges_filtered.csv', index=False)
nodes.to_csv("nodes_filtered.csv", index=False)